
# Classificação de Risco de Câncer com Rede Neural Profunda (TensorFlow/Keras)

Este notebook implementa um modelo de **Rede Neural Artificial profunda**, utilizando **TensorFlow/Keras**, para o problema de **classificação de risco de câncer**.

Principais pontos:

- Leitura do dataset a partir de **upload de arquivo CSV**.
- **Limpeza e pré-processamento** dos dados (numéricos e categóricos).
- Codificação das classes com `LabelEncoder`.
- **Validação cruzada estratificada k-fold** (k = 10).
- Treinamento de um novo modelo Keras em **cada fold**.
- Cálculo da **acurácia média** e desvio padrão entre os folds.

Requisito da disciplina: obter **acurácia média ≥ 0.70** na validação cruzada.


## 1. Importação de bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import io
import random

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf

import ipywidgets as widgets
from IPython.display import display


## 2. Reprodutibilidade (semente aleatória)

In [ ]:
SEED = 42

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


## 3. Upload do arquivo CSV


Faça o upload do arquivo **`classificacao_risco_cancer_varios_valores.csv`** no widget abaixo.


In [ ]:
uploader = widgets.FileUpload(accept='.csv', multiple=False)
display(uploader)


### 3.1 Leitura do dataset enviado

In [ ]:
if len(uploader.value) == 0:
    raise ValueError("Nenhum arquivo enviado. Faça o upload do CSV no widget acima.")

# Extrair conteúdo do arquivo
file_info = list(uploader.value.values())[0]
file_content = file_info['content']

# Ler CSV a partir de bytes
df = pd.read_csv(io.BytesIO(file_content))

print("Formato do dataset:", df.shape)
df.head()


## 4. Análise exploratória inicial

In [ ]:
print("Tipos de dados:")
print(df.dtypes)

print("\nValores ausentes por coluna:")
print(df.isna().sum())


In [ ]:
# Estatísticas numéricas
df.describe(include=[np.number])


In [ ]:
# Estatísticas categóricas
df.describe(include=['object'])



## 5. Limpeza dos dados e definição da variável alvo

Passos adotados:

1. Substituição de strings que representam valores ausentes (`"NaN"`, `"nan"`, `"None"`, `"?"`, etc.) por `NaN` real.
2. Remoção da coluna de identificador (`Patient Id`), se existir.
3. Definição de **`Level`** como variável-alvo.
4. Remoção de linhas em que `Level` é ausente.


In [ ]:
dados = df.copy()

# Strings que representam valores nulos
valores_nulos_str = ["NaN", "nan", "NONE", "None", "?", "null", "NULL", " "]
dados = dados.replace(valores_nulos_str, np.nan)

# Remover coluna de identificador, se presente
if "Patient Id" in dados.columns:
    dados = dados.drop(columns=["Patient Id"])

if "Level" not in dados.columns:
    raise ValueError("A coluna 'Level' não foi encontrada no dataset. Verifique o nome da variável-alvo.")

# Remover linhas sem rótulo
dados = dados.dropna(subset=["Level"])

X = dados.drop(columns=["Level"])
y = dados["Level"]

print("Formato de X:", X.shape)
print("Formato de y:", y.shape)

print("\nDistribuição das classes em 'Level':")
print(y.value_counts(normalize=True))


## 6. Separação de variáveis numéricas e categóricas

In [ ]:
colunas_numericas = X.select_dtypes(include=[np.number]).columns.tolist()
colunas_categoricas = X.select_dtypes(include=['object']).columns.tolist()

print("Variáveis numéricas:", colunas_numericas)
print("Variáveis categóricas:", colunas_categoricas)



## 7. Pré-processamento dos atributos

Será construído um **ColumnTransformer** para aplicar transformações distintas:

- Atributos numéricos:
  - `SimpleImputer(strategy="median")`
  - `StandardScaler()`

- Atributos categóricos:
  - `SimpleImputer(strategy="most_frequent")`
  - `OneHotEncoder(handle_unknown="ignore")`


In [ ]:
from sklearn.pipeline import Pipeline

transformador_numerico = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

transformador_categorico = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessador = ColumnTransformer(
    transformers=[
        ("num", transformador_numerico, colunas_numericas),
        ("cat", transformador_categorico, colunas_categoricas)
    ]
)
preprocessador


## 8. Codificação da variável alvo (`LabelEncoder`)

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Classes encontradas:", list(label_encoder.classes_))
print("Exemplo de codificação:")
for classe, codigo in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"{classe} -> {codigo}")


## 9. Definição do modelo Keras (rede neural profunda)

In [ ]:
def build_model(input_dim, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Teste rápido da função (apenas para ver o resumo depois de conhecido o input_dim)
print("Função de construção de modelo definida.")



## 10. Validação cruzada estratificada k-fold com Keras

Configurações:

- `k = 10` folds.
- Para cada fold:
  1. Ajusta-se o **pré-processador** apenas nos dados de treino do fold.
  2. Transforma-se treino e validação.
  3. Constrói-se um **novo modelo Keras**.
  4. Treina-se o modelo no fold de treino.
  5. Avalia-se a acurácia no fold de validação.
- Ao final, calcula-se a **acurácia média** e o **desvio padrão**.


In [ ]:
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=SEED)

scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_encoded), start=1):
    print(f"\n===== Fold {fold}/{k} =====")
    X_train_raw, X_val_raw = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]

    # Ajustar o pré-processador SOMENTE nos dados de treino
    X_train_proc = preprocessador.fit_transform(X_train_raw)
    X_val_proc = preprocessador.transform(X_val_raw)

    input_dim = X_train_proc.shape[1]
    num_classes = len(label_encoder.classes_)

    # Calcular pesos de classe para lidar com desbalanceamento (opcional, mas recomendado)
    class_weights_array = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(y_train),
        y=y_train
    )
    class_weights = {i: w for i, w in enumerate(class_weights_array)}

    # Construir e treinar o modelo
    model = build_model(input_dim, num_classes)

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )

    # Avaliação no fold
    loss, acc = model.evaluate(X_val_proc, y_val, verbose=0)
    print(f"Acurácia no fold {fold}: {acc:.4f}")
    scores.append(acc)

scores = np.array(scores)
print("\nAcurácias por fold (10-fold):")
print(scores)
print("\nAcurácia média: {:.4f}".format(scores.mean()))
print("Desvio padrão: {:.4f}".format(scores.std()))



## 11. Discussão dos resultados

- A lista de acurácias por fold mostra o desempenho do modelo em cada uma das partições.
- A **acurácia média** é a métrica principal de interesse.
- O **desvio padrão** indica a estabilidade do modelo entre diferentes divisões do conjunto de dados.

Se a acurácia média for **≥ 0.70**, o requisito do trabalho é atendido.  
Caso contrário, é possível explorar:

- Arquiteturas mais profundas (mais camadas ou mais neurônios).
- Ajuste fino de hiperparâmetros (taxa de aprendizado, batch size, número de épocas).
- Estratégias adicionais de balanceamento de classes.


## 12. Experimentos adicionais (opcional)

In [ ]:
# Espaço para testar novas arquiteturas, hiperparâmetros ou estratégias de pré-processamento.

# Exemplo (rascunho):
# def build_model_v2(input_dim, num_classes):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Input(shape=(input_dim,)),
#         tf.keras.layers.Dense(256, activation='relu'),
#         tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dropout(0.3),
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dense(num_classes, activation='softmax')
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
#         loss='sparse_categorical_crossentropy',
#         metrics=['accuracy']
#     )
#     return model
